- 9/7 NIHSmtg用にカテゴリ数, カテゴリ詳細を付与した後fの列を消す等してxlsxを整形
- カテゴリ数についてのヒストグラムを作成

In [ ]:
%matplotlib inline

import pandas as pd
import numpy as np
from IPython.display import display
import sys
from tqdm import tqdm_notebook as tqdm
import matplotlib.pyplot as plt
import seaborn as sns
import re #正規表現
import pickle
import warnings
import os

from sklearn.model_selection import train_test_split, GridSearchCV, StratifiedKFold
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import confusion_matrix, f1_score, roc_curve, roc_auc_score, precision_score, recall_score, accuracy_score
from sklearn import svm

from multiprocessing import Process, Pool

# from protein_utility.protein_utility import *
from protein_utility_test import *
import protein_utility.parameters as parameters

import xml.etree.ElementTree as ET
from sklearn.preprocessing import StandardScaler, MinMaxScaler


sns.set()

warnings.filterwarnings('ignore') #警告無視注意


SEED=1

pd.set_option("display.max_columns", 100)

aa_list =["A","B","C","D","E","F","G","H","I","K","L","M","N","P","Q","R","S","T","U","V","W","X","Y","Z"]
#20種+4
categoryList = ["Bovine", "Buckwheat", "Chicken", "Crab", "Kiwi", "Peanut", "Salmon", "Shrimp", "Soybean", "Wheat"]

OUTPUT_DIR = 'output_ipynb/04_output_fig4e_supData4'
os.makedirs(OUTPUT_DIR, exist_ok=True)

In [ ]:
# create_Xにおいてはstd, normalize絡みのオプションを指定しなくていい

# FastWY
params = parameters.Parameters(FOOD_WITH_MTEC_ORDER = True, op_tail = "C1GT1L1800R10k", Jan2021 = True)

# Allerdictor
# params = parameters.Parameters(FOOD_WITH_MTEC_ORDER = True, op_tail = "allerdictor", ALLERDICTOR = True, allerdictor_min_sup=8)

# MEME
# maxw=30
# nmotifs=300
# meme_op = "food_order_aller_nmotifs{}maxw{}".format(nmotifs, maxw)
# params = parameters.Parameters(FOOD_WITH_MTEC_ORDER = True, SIM=False, MEME = True, op_tail = "mtec", meme_op = meme_op, meme_pattern_mode = True)


p=20 #プロセス数


# categoryList = ["All"]

# カテゴリ数とカテゴリ詳細を追記

In [ ]:
params_ps1_a001 = parameters.Parameters(FOOD_ONLY=True, op_tail = "C1a001L1800R10k")
params_ps1_a005 = parameters.Parameters(FOOD_ONLY=True, op_tail = "C1L1800R10k")
params_ps1_a01 = parameters.Parameters(FOOD_ONLY=True, op_tail = "C1a01L1800R10k")

params_ps2_a001 = parameters.Parameters(FOOD_ORDER=True, op_tail = "C1Ga001L1800R10k")
params_ps2_a005 = parameters.Parameters(FOOD_ORDER=True, op_tail = "C1GL1800R10k")
params_ps2_a01 = parameters.Parameters(FOOD_ORDER=True, op_tail = "C1Ga01L1800R10k")

params_ps3_a001 = parameters.Parameters(FOOD_ONLY_WITH_MTEC_ORDER=True, op_tail = "C1Ga001L1800R10k")
params_ps3_a005 = parameters.Parameters(FOOD_ONLY_WITH_MTEC_ORDER=True, op_tail = "C1GL1800R10k")
params_ps3_a01 = parameters.Parameters(FOOD_ONLY_WITH_MTEC_ORDER=True, op_tail = "C1Ga01L1800R10k")

params_ps4_a001 = parameters.Parameters(FOOD_WITH_MTEC_ORDER=True, op_tail = "C1Ga001T1L1800R10k", Jan2021 = True)
params_ps4_a005 = parameters.Parameters(FOOD_WITH_MTEC_ORDER=True, op_tail = "C1GT1L1800R10k", Jan2021=True)
params_ps4_a01 = parameters.Parameters(FOOD_WITH_MTEC_ORDER=True, op_tail = "C1Ga01T1L1800R10k", Jan2021=True)

In [ ]:
#ps_list = [1, 2, 3, 4]
ps_list = [4]

#alpha_list = ["001", "005", "01"]
alpha_list = ["001", "005"]

params_dict = {1: {"001": params_ps1_a001, "005": params_ps1_a005, "01": params_ps1_a01}, 2: {"001": params_ps2_a001, "005": params_ps2_a005, "01": params_ps2_a01}, 3: {"001": params_ps3_a001, "005": params_ps3_a005, "01": params_ps3_a01}, 4: {"001": params_ps4_a001, "005": params_ps4_a005, "01": params_ps4_a01}}

for ps in ps_list:
    for alpha_l in alpha_list:
        for extract_label, extract_stat in {1: 'allergen'}.items(): # Added by Hiroyuki Hanada, 2021.6.17
        #for extract_label, extract_stat in {1: 'allergen', -1: 'nonallergen'}.items(): # Added by Hiroyuki Hanada, 2021.6.17
            fname_supData4 = OUTPUT_DIR + ("/supData4_{}_ps{}_a{}_Jan2021.xlsx".format(extract_stat, ps, alpha_l))
            params = params_dict[ps][alpha_l]
            category = "all"

            df_data = pd.read_csv(params.def_aa(category, train_test="train"), index_col=0)
            df_aa_result = pd.read_csv(params.def_aa_result(category), index_col=0, keep_default_na=False)
            df_aa_result = df_aa_result[df_aa_result["label"] ==extract_label].reset_index(drop=True) # アレルゲンパターンのみ注意
            print("df_aa: ", df_aa_result.shape)
            df_aa_result["category_num"] = ""
            df_aa_result["category"] = ""

            for i_pat, pat in tqdm(enumerate(df_aa_result["pattern"])):
                categ_set = set()
                categ_num = 0
                for i_seq, seq in enumerate(df_data["sequence"]):
                    if not isinstance(pat, str):
                        raise RuntimeError("`pat` is not a string, but %s (%s)" % (str(type(pat)), str(pat)))
                    if pat in seq:
                        categ = df_data.at[i_seq, "bio_category"]
                        if categ == "Food":
                            categ = df_data.at[i_seq, "category"]
                        if not categ in categ_set and categ == categ:
                            categ_set.add(categ)
                            categ_num += 1
                df_aa_result.at[i_pat, "category_num"] = categ_num
                df_aa_result.at[i_pat, "category"] = categ_set


            # 整形
            df_aa_result = df_aa_result[["pattern", "length", "adjusted_p", "supportSum", "sup+", "sup-", "category_num", "category"]]
            display(df_aa_result)

            df_aa_result.to_excel(fname_supData4)
        

# カテゴリ数のヒストグラム
実装終わり次第fig4作成スクリプトの方で行う

In [ ]:
scale=5
alpha=0.6
bins=20
font_size=15
target_label="allergen"
#target_label="nonallergen"

for ps in ps_list:
    params_str="ps%d" % ps
    for alpha_l in alpha_list:
        fname_supData4 = OUTPUT_DIR + ("/supData4_{}_ps{}_a{}_Jan2021.xlsx".format(target_label, ps, alpha_l))
        df_aa_result_plus = pd.read_excel(fname_supData4, index_col=0)
        display(df_aa_result_plus)
        fig = plt.figure(figsize=(2*scale, 1*scale))
        # fig.subplots_adjust(wspace=0.2, hspace=0.4)

        ax = fig.add_subplot(1, 1, 1)

        ax.hist(list(df_aa_result_plus["category_num"]), bins=bins, alpha=alpha, label="plus_pattern", color = "red")

        ax.set_title("Number of included categories ({}, {}, alpha={})".format(params_str, category, alpha_l), fontsize=font_size)
        ax.set_xlabel("Number of included categories", fontsize=font_size)
        ax.set_ylabel("Number of patterns", fontsize=font_size)
        ax.legend()

        #fig.savefig(OUTPUT_DIR + ("/hist_pattern_all_{}_{}_{}_{}_Jan2021.pdf".format(target_label, params_str, alpha_l, "category_num")), bbox_inches="tight")
        fig.savefig(OUTPUT_DIR + ("/figure4e_hist_pattern_all_{}_{}_{}_{}_Jan2021.pdf".format(target_label, params_str, alpha_l, "category_num")), bbox_inches="tight")
        fig.show()